In [1]:
import pandas as pd
import os
import random
from tqdm import tqdm
from pprint import pprint
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.metrics.pairwise import linear_kernel

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

from numpy import dot 
from numpy.linalg import norm

from transformers import AutoTokenizer

In [14]:
file_list = os.listdir('./data')
print(file_list, len(file_list), 2500*len(file_list))

['이어폰_data.csv', '노트북_data.csv', '모니터_data.csv', '데스크탑_data.csv', '대형가전_data.csv', '필름카메라_data.csv', '웨어러블_data.csv', 'dslr_data.csv', '스피커_data.csv', '일반폰_data.csv', 'tv_data.csv', '사무기기_data.csv', '키보드마우스_data.csv', '스마트폰_data.csv', '테블릿_data.csv'] 15 37500


In [15]:
data = pd.DataFrame()
data_len = 0
for file in file_list:
    d = pd.read_csv('./data/' + file, lineterminator='\n')
    d['category'] = file.split('_')[0]
    data_len += len(d)
    print(data_len)
    data = pd.concat([data, d])

2500
5000
7500
10000
12500
15000
17500
20000
22500
24886
27386
28315
30815
33315
35815


In [16]:
len(data), data.columns

(35815,
 Index(['Unnamed: 0', 'title', 'tag', 'location', 'description', 'img',
        'image_count', 'pid', 'url', 'category'],
       dtype='object'))

In [20]:
MODEL_NAME = 'klue/bert-base'
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

In [21]:
def cos_sim(a, b): 
    return dot(a, b)/(norm(a)*norm(b))


def make_matrix(feats, list_data):
    freq_list = []
    for feat in feats:
        freq = 0 
        for word in list_data: 
            if feat == word: 
                freq += 1 
        freq_list.append(freq) 
    return freq_list

In [22]:
df_output = pd.DataFrame(columns=['index', 'title', 'context', 'hashtag', 'category'])

for i in tqdm(range(len(data))):
    row = data.iloc[i]
    title_text = row['title']
    context_text = [d for d in row['description'].split('\n') if len(d) > 2]
    
    # print('title : ', title_text) 
    # print('hashtag : ', row['tag'])
    # print()
    
    result_context = ''
    title = tokenizer.convert_ids_to_tokens(tokenizer(title_text)['input_ids'])[1:-1]
    for origin_text in context_text:
        context = tokenizer.convert_ids_to_tokens(tokenizer(origin_text)['input_ids'])[1:-1]
        
        title_vector = make_matrix(title, title)
        # print(len(title), len(context))
        # print(title)
        # print(context)
        sim = make_matrix(title, context) 
        # print(title_vector)
        similarity = cos_sim(title_vector, sim)
        # print(origin_text, '\t', similarity)

        if similarity > 0.3:
            result_context += origin_text
    
    if result_context:    
        df_output = df_output.append(pd.DataFrame([[ i, title_text, result_context, row['tag'], row['category'] ]],
                                        columns=['index', 'title', 'context', 'hashtag', 'category']),
                                        ignore_index=True)
        
    # print()
    # print('=============================')
    # print()
    

  0%|          | 0/35815 [00:00<?, ?it/s]<ipython-input-21-c7179cb779ae>:2: RuntimeWarning: invalid value encountered in true_divide
  return dot(a, b)/(norm(a)*norm(b))
100%|██████████| 35815/35815 [03:24<00:00, 174.99it/s]


In [24]:
len(df_output)

28909

In [23]:
df_output.to_csv('./all_data.csv')